In [49]:
from enhanced_technical_introspector import TechnicalIntrospector

introspector = TechnicalIntrospector(system)
introspector.generate_report(quick_mode=False)

# # Or specific diagnostics:
# introspector.diagnose_current_state()
# introspector.profile_performance()
# introspector.check_data_freshness()

✅ Technical diagnostic report saved: ./docs/TECHNICAL_DIAGNOSTIC.md
   Issues found: 0
   Warnings: 0


# 🔧 Technical System Diagnostic Report

*Generated: 2025-11-07 22:06:43*
*Quick Mode: OFF*

---

## 1. 🏥 System Health Check

**Overall Status**: ✅ HEALTHY

### Component Status
| Component | Status | Details |
|-----------|--------|---------|
| data | ✅ | 3769 rows |
| model | ✅ | Trained |
| detector_coverage | ✅ | 0 detectors with <70% coverage |
| data_freshness | ✅ | 3 days old |
| data_completeness | ✅ | 79.1% complete |

## 2. 🎛️ Model Configuration Deep Dive

### Anomaly Detector Configuration
```python
contamination: 0.050
n_estimators: 100
max_samples: auto
random_state: 42
```

### Individual Detector Parameters
| Detector | Features Used | Coverage | Active |
|----------|--------------|----------|--------|
| vix_mean_reversion | 18 | 100.0% | ✅ |
| vix_momentum | 18 | 100.0% | ✅ |
| vix_regime_structure | 18 | 100.0% | ✅ |
| cboe_options_flow | 39 | 100.0% | ✅ |
| cboe_cross_dynamics | 17 | 100.0% | ✅ |
| vix_spx_relationship | 17 | 100.0% | ✅ |
| spx_price_action | 15 | 75.0% | ✅ |
| spx_volatility_regime | 20 | 90.9% | ✅ |
| cross_asset_divergence | 23 | 88.5% | ✅ |
| tail_risk_complex | 15 | 83.3% | ✅ |
| futures_term_structure | 27 | 100.0% | ✅ |
| macro_regime_shifts | 19 | 100.0% | ✅ |
| momentum_acceleration | 20 | 100.0% | ✅ |
| percentile_extremes | 19 | 100.0% | ✅ |
| random_4 | 0 | 0.0% | ✅ |

## 3. 🔄 Data Pipeline Flow Trace

### Data Sources → Features → Models
```
Data Fetching...........................       ✅ OK
  ↳ VIX: 3769 observations
  ↳ SPX: 3769 observations
Feature Engineering.....................       ✅ OK
  ↳ Generated 696 features
  ↳ Time period: 3769 days
Model Training..........................       ✅ OK
  ↳ 15/15 detectors trained
  ↳ Ensemble scores computed: 3769
```

### Feature Generation Summary
- **Raw Market Data Points**: 7538
- **Engineered Features**: 696
- **Final Feature Set**: 696
- **Data Reduction Ratio**: 0.1x

## 4. 📅 Data Freshness & Staleness

### Last Update Times
| Data Source | Last Update | Age (days) | Status |
|-------------|-------------|------------|--------|
| Main Features | 2025-11-04 | 3.9 | ❌ |

### ⚠️ Stale Features (>5% missing in recent data)
- **SKEW**: 100.0% missing
- **SKEW_change_21d**: 100.0% missing
- **SKEW_zscore_63d**: 100.0% missing
- **PCCI**: 100.0% missing
- **PCCI_change_21d**: 100.0% missing
- **PCCI_zscore_63d**: 100.0% missing
- **PCCE**: 100.0% missing
- **PCCE_change_21d**: 100.0% missing
- **PCCE_zscore_63d**: 100.0% missing
- **PCC**: 100.0% missing

## 5. 🎯 Current Anomaly Detection Breakdown

### Ensemble Score: 38.1%

### Detector Contributions
| Detector | Score | Weight | Weighted Score | Agreement |
|----------|-------|--------|----------------|-----------|
| spx_price_action | 81.1% | 1.00 | 81.1% | 🔴 |
| tail_risk_complex | 79.9% | 1.00 | 79.9% | 🔴 |
| spx_volatility_regime | 59.1% | 1.00 | 59.1% | 🔴 |
| vix_momentum | 55.8% | 1.00 | 55.8% | 🟢 |
| vix_regime_structure | 52.0% | 1.00 | 52.0% | 🟢 |
| percentile_extremes | 43.4% | 1.00 | 43.4% | 🟢 |
| cross_asset_divergence | 38.4% | 1.00 | 38.4% | 🟢 |
| vix_mean_reversion | 37.9% | 1.00 | 37.9% | 🟢 |
| vix_spx_relationship | 31.3% | 1.00 | 31.3% | 🟢 |
| macro_regime_shifts | 13.4% | 1.00 | 13.4% | 🔴 |
| momentum_acceleration | 8.2% | 1.00 | 8.2% | 🔴 |
| cboe_cross_dynamics | 7.7% | 1.00 | 7.7% | 🔴 |
| futures_term_structure | 5.4% | 1.00 | 5.4% | 🔴 |
| cboe_options_flow | 1.5% | 1.00 | 1.5% | 🔴 |

### Top Features Driving Current Anomaly
| Feature | Importance | Current Value | Z-Score |
|---------|-----------|---------------|---------|
| spx_vs_ma200 | 0.091 | 10.72 | 1.00 |
| spx_lag1 | 0.088 | 6851.97 | 2.70 |
| spx_ret_5d | 0.082 | -1.73 | -0.89 |
| spx_lag5 | 0.082 | 6890.89 | 2.74 |
| spx_ret_13d | 0.075 | 2.15 | 0.43 |
| spx_ret_63d | 0.070 | 6.72 | 0.54 |
| rsi_14 | 0.067 | 59.06 | 0.14 |
| spx_momentum_z_10d | 0.067 | -0.48 | -0.34 |
| spx_momentum_z_21d | 0.062 | -1.57 | -1.16 |
| bb_width_20d | 0.058 | 5.64 | -0.13 |

## 6. 🔗 Feature Correlation & Redundancy Analysis

### High Correlation Pairs (>95%)
| Feature 1 | Feature 2 | Correlation |
|-----------|-----------|-------------|
| 1M_Treasury_zscore_252d | Breakeven_Inflation_10Y_zscore_252d | 1.000 |
| 1M_Treasury_zscore_252d | SOFR_90D_level | 1.000 |
| 1M_Treasury_zscore_252d | Corporate_Master_OAS_zscore_252d | 1.000 |
| 1M_Treasury_zscore_252d | High_Yield_OAS_level | 1.000 |
| 1M_Treasury_zscore_252d | CCC_High_Yield_OAS_level | 1.000 |
| 1M_Treasury_zscore_252d | 7Y_Treasury_level | 1.000 |
| 1M_Treasury_zscore_252d | BB_High_Yield_OAS_level | 1.000 |
| 1M_Treasury_zscore_252d | Yield_Curve_10Y3M_zscore_252d | 1.000 |
| 1M_Treasury_zscore_252d | 20Y_Treasury_zscore_63d | 1.000 |
| 1M_Treasury_zscore_252d | 3M_Treasury_zscore_252d | 1.000 |
| 1M_Treasury_zscore_252d | 3Y_Treasury_level | 1.000 |
| 1M_Treasury_zscore_252d | 20Y_Treasury_zscore_252d | 1.000 |
| 1M_Treasury_zscore_252d | Corporate_Master_OAS_zscore_63d | 1.000 |
| 1M_Treasury_zscore_252d | 2Y_Treasury_zscore_252d | 1.000 |
| 1M_Treasury_zscore_252d | Breakeven_Inflation_10Y_zscore_63d | 1.000 |

**Recommendation**: Consider removing 17552 redundant features to improve performance.

## 7. ⚡ Performance Profiling

### Execution Time Breakdown
| Operation | Time (ms) | % of Total |
|-----------|-----------|------------|
| batch_10_detections | 168.6 | 89.7% |
| single_detection | 19.3 | 10.3% |

## 8. 🔍 Common Failure Modes & Solutions

✅ No common failure modes detected.

## 9. 💡 What-If Scenario Analysis

### VIX Spike to 40
**Scenario**: If VIX suddenly spikes to 40 (crisis level)
**Expected Behavior**: Ensemble score would likely exceed 93% (CRITICAL threshold)
**Current System Response**: Multiple detectors would fire: vix_regime_structure, vix_momentum, cross_asset_divergence

### SKEW >150
**Scenario**: If SKEW index exceeds 150 (extreme tail risk)
**Expected Behavior**: tail_risk_complex detector triggers, ensemble score elevates
**Current System Response**: If SKEW features are available, system would classify as HIGH/CRITICAL

### All CBOE Data Missing
**Scenario**: If CBOE features become unavailable
**Expected Behavior**: System continues to function with reduced capability
**Current System Response**: 5/15 detectors would be disabled, ensemble relies on VIX/SPX/futures detectors

## 10. 📊 Data Quality Heatmap

### Feature Quality by Category
| Category | Total Features | Complete | Sparse | Missing | Quality Score |
|----------|---------------|----------|--------|---------|---------------|
| VIX | 78 | 73 | 0 | 5 | 93.6% |
| SPX | 47 | 44 | 0 | 3 | 93.6% |
| CBOE | 232 | 24 | 33 | 175 | 17.5% |
| Futures | 45 | 35 | 10 | 0 | 88.9% |
| Macro | 14 | 11 | 1 | 2 | 82.1% |
| Meta | 280 | 84 | 122 | 74 | 51.8% |

## 11. 🚀 System Optimization Recommendations

### 🟢 Optimization Opportunities
- Remove 17552 highly correlated features to reduce redundancy
- Investigate 197 stale features with high recent missing data
- Consider adding feature selection to reduce dimensionality
- Implement caching for expensive feature calculations
- Add monitoring alerts for data freshness

## 12. 📖 Quick Reference: Troubleshooting Guide


### Common Issues & Quick Fixes

**Issue**: System says "data too old"
- **Check**: `system.orchestrator.features.index[-1]`
- **Fix**: Run `system.refresh()` or retrain with fresh data

**Issue**: Ensemble score always near 0% or 100%
- **Check**: Are thresholds computed? `system.orchestrator.anomaly_detector.statistical_thresholds`
- **Fix**: Retrain system to recalculate thresholds

**Issue**: Many detectors show 0% coverage
- **Check**: CBOE files in `./CBOE_Data_Archive/`
- **Fix**: Download CBOE historical data or disable CBOE features in config

**Issue**: "Core data fetch failed" error
- **Check**: Internet connection, yfinance API status
- **Fix**: Run `system.orchestrator.fetcher.fetch_core_data(...)` separately to debug

**Issue**: High memory usage
- **Check**: Feature matrix size with `system.orchestrator.features.memory_usage(deep=True).sum()`
- **Fix**: Reduce training window in config.py (TRAINING_YEARS)

**Issue**: Slow detection speed (>1 second)
- **Check**: Number of features and detectors active
- **Fix**: Reduce features, disable low-value detectors, or enable quick_mode

**Issue**: NaN/Inf values in features
- **Check**: `system.orchestrator.features.isnull().sum()` and `np.isinf(system.orchestrator.features).sum()`
- **Fix**: Review feature_engine.py for division by zero or missing data handling


---
*Report generated in 2.34 seconds*

'# 🔧 Technical System Diagnostic Report\n\n*Generated: 2025-11-07 22:06:43*\n*Quick Mode: OFF*\n\n---\n\n## 1. 🏥 System Health Check\n\n**Overall Status**: ✅ HEALTHY\n\n### Component Status\n| Component | Status | Details |\n|-----------|--------|---------|\n| data | ✅ | 3769 rows |\n| model | ✅ | Trained |\n| detector_coverage | ✅ | 0 detectors with <70% coverage |\n| data_freshness | ✅ | 3 days old |\n| data_completeness | ✅ | 79.1% complete |\n\n## 2. 🎛️ Model Configuration Deep Dive\n\n### Anomaly Detector Configuration\n```python\ncontamination: 0.050\nn_estimators: 100\nmax_samples: auto\nrandom_state: 42\n```\n\n### Individual Detector Parameters\n| Detector | Features Used | Coverage | Active |\n|----------|--------------|----------|--------|\n| vix_mean_reversion | 18 | 100.0% | ✅ |\n| vix_momentum | 18 | 100.0% | ✅ |\n| vix_regime_structure | 18 | 100.0% | ✅ |\n| cboe_options_flow | 39 | 100.0% | ✅ |\n| cboe_cross_dynamics | 17 | 100.0% | ✅ |\n| vix_spx_relationship | 17 | 10

In [65]:
#!/usr/bin/env python3
"""
Debug script to diagnose backfill issues
Run this to see exactly what's happening with VIX data
"""

import sqlite3
import pandas as pd
import yfinance as yf
from pathlib import Path

# Database path
db_path = Path("data_cache/predictions.db")

# Connect and get predictions needing backfill
conn = sqlite3.connect(db_path)
query = """
SELECT prediction_id, forecast_date, horizon, current_vix
FROM forecasts
WHERE actual_vix_change IS NULL
LIMIT 5
"""

predictions = pd.read_sql_query(query, conn, parse_dates=['forecast_date'])
conn.close()

print("=" * 80)
print("DEBUGGING BACKFILL ISSUES")
print("=" * 80)
print(f"\nFound {len(predictions)} predictions to backfill")
print("\nPrediction details:")
print(predictions)

# Fetch VIX data
print("\n" + "=" * 80)
print("FETCHING VIX DATA")
print("=" * 80)

vix_df = yf.download("^VIX", progress=False)
print(f"\nDownloaded VIX data:")
print(f"  Type: {type(vix_df)}")
print(f"  Shape: {vix_df.shape if hasattr(vix_df, 'shape') else 'N/A'}")
print(f"  Columns: {vix_df.columns.tolist() if hasattr(vix_df, 'columns') else 'N/A'}")

# Extract Close column
if isinstance(vix_df, pd.DataFrame):
    if 'Close' in vix_df.columns:
        vix_series = vix_df['Close']
    else:
        vix_series = vix_df.iloc[:, 0]  # First column
else:
    vix_series = vix_df

# Ensure Series
if isinstance(vix_series, pd.DataFrame):
    vix_series = vix_series.squeeze()

print(f"\nExtracted VIX series:")
print(f"  Type: {type(vix_series)}")
print(f"  Length: {len(vix_series)}")
print(f"  Index type: {type(vix_series.index)}")
print(f"  Sample values:")
print(vix_series.tail())

# Test lookup for each prediction
print("\n" + "=" * 80)
print("TESTING VIX LOOKUPS")
print("=" * 80)

for idx, pred in predictions.iterrows():
    forecast_date = pred['forecast_date']
    horizon = pred['horizon']
    target_date = forecast_date + pd.Timedelta(days=horizon)
    
    print(f"\nPrediction {idx + 1}:")
    print(f"  Forecast date: {forecast_date.date()}")
    print(f"  Target date: {target_date.date()}")
    print(f"  Horizon: {horizon} days")
    
    # Try exact lookup
    if target_date in vix_series.index:
        result = vix_series.loc[target_date]
        print(f"  ✅ Exact match found")
        print(f"     Result type: {type(result)}")
        print(f"     Result value: {result}")
        
        # Try to convert to float
        try:
            if isinstance(result, pd.Series):
                if len(result) == 1:
                    value = float(result.iloc[0])
                    print(f"     ✅ Converted to float: {value}")
                else:
                    print(f"     ❌ Multiple values: {len(result)}")
            else:
                value = float(result)
                print(f"     ✅ Converted to float: {value}")
        except Exception as e:
            print(f"     ❌ Conversion failed: {e}")
    else:
        print(f"  ⚠️  Exact match not found, trying asof...")
        result = vix_series.asof(target_date)
        print(f"     Result type: {type(result)}")
        print(f"     Result value: {result}")
        
        # Try to convert
        try:
            if isinstance(result, pd.Series):
                if len(result) == 1:
                    value = float(result.iloc[0])
                    print(f"     ✅ Converted to float: {value}")
                else:
                    print(f"     ❌ Multiple values: {len(result)}")
            else:
                value = float(result)
                print(f"     ✅ Converted to float: {value}")
        except Exception as e:
            print(f"     ❌ Conversion failed: {e}")

print("\n" + "=" * 80)
print("DIAGNOSIS COMPLETE")
print("=" * 80)
print("\nIf you see '❌ Multiple values' or conversion errors above,")
print("that's the source of the 'ambiguous Series' error.")
print("\nThe fix needs to handle the exact data structure returned by yfinance.")

DEBUGGING BACKFILL ISSUES

Found 5 predictions to backfill

Prediction details:
                          prediction_id forecast_date  horizon  current_vix
0  4182fda2-b6b2-4431-8c60-ace1d6c90543    2025-11-10        5        18.01
1  30649c8c-0f0f-452d-81a3-a07e3282a1b4    2025-11-10        5        18.01
2  ef51c1a5-1edb-4b8a-a781-52bfb35c4f41    2025-11-10        5        18.01
3  529dc6d0-7197-4bd1-88eb-feea6d1cffaa    2024-01-07        5        13.20
4  3eb0818d-10ae-45da-a316-a91dc2a67a77    2024-01-08        5        14.04

FETCHING VIX DATA

Downloaded VIX data:
  Type: <class 'pandas.core.frame.DataFrame'>
  Shape: (22, 5)
  Columns: [('Close', '^VIX'), ('High', '^VIX'), ('Low', '^VIX'), ('Open', '^VIX'), ('Volume', '^VIX')]

Extracted VIX series:
  Type: <class 'pandas.core.series.Series'>
  Length: 22
  Index type: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
  Sample values:
Date
2025-11-04    19.00
2025-11-05    18.01
2025-11-06    19.50
2025-11-07    19.08
2025-1

In [37]:
# Quick diagnostic script
import sqlite3
import pandas as pd

conn = sqlite3.connect('data_cache/predictions.db')

# Check prediction dates
df = pd.read_sql_query("""
    SELECT 
        forecast_date,
        horizon,
        actual_vix_change IS NOT NULL as has_actual,
        date(forecast_date, '+' || horizon || ' days') as target_date
    FROM forecasts
    ORDER BY forecast_date DESC
    LIMIT 100
""", conn, parse_dates=['forecast_date', 'target_date'])

print(df)
print(f"\nToday: {pd.Timestamp.now().date()}")
print(f"Predictions with actuals: {df['has_actual'].sum()}/{len(df)}")
    FROM forecasts

                           prediction_id                   timestamp  \
0   4182fda2-b6b2-4431-8c60-ace1d6c90543  2025-11-10T12:26:03.836832   
1   0de2fbea-17a4-4d09-a139-b103c25a17cb  2025-11-10T13:52:39.450119   
2   3981008d-714b-4639-9caf-226e07ecc3b4  2025-11-10T13:52:33.630141   
3   16eec5af-8bc3-4927-b9a1-f008ba10113a  2025-11-10T21:27:03.018547   
4   194a4504-844b-4155-9aeb-00b05fe15fbc  2025-11-10T21:26:56.875460   
..                                   ...                         ...   
70  087515af-12fb-439d-9c00-6849dd62f4f2  2025-11-10T13:29:34.781666   
71  7c9e58cb-48fe-4898-acd9-cd81397a5751  2025-11-10T13:28:52.294450   
72  f852b8fd-6527-40ad-8dff-d61ad80d28d2  2025-11-10T13:28:46.147887   
73  3eb0818d-10ae-45da-a316-a91dc2a67a77  2025-11-10T13:28:39.749458   
74  529dc6d0-7197-4bd1-88eb-feea6d1cffaa  2025-11-10T13:28:33.003388   

   forecast_date  horizon       calendar_cohort  cohort_weight  \
0     2025-11-10        5             mid_cycle            1.0   
1  

KeyError: 'has_actual'

In [35]:
#!/usr/bin/env python3
"""
Standalone script to clean duplicate predictions and show database stats.
Run from src/ directory: python cleanup_duplicates.py
"""

import sys
from pathlib import Path

# Ensure we can import from current directory


import sqlite3
import pandas as pd

DB_PATH = "data_cache/predictions.db"

def remove_duplicates():
    """Remove duplicate predictions, keeping the earliest by timestamp."""
    conn = sqlite3.connect(DB_PATH)
    
    print("\n" + "=" * 80)
    print("DUPLICATE CLEANUP")
    print("=" * 80)
    
    # Find duplicates
    cursor = conn.execute("""
        SELECT forecast_date, horizon, COUNT(*) as count
        FROM forecasts
        GROUP BY forecast_date, horizon
        HAVING count > 1
    """)
    
    duplicates = cursor.fetchall()
    
    if len(duplicates) == 0:
        print("✅ No duplicates found")
        conn.close()
        return 0
    
    print(f"⚠️  Found {len(duplicates)} duplicate date-horizon pairs\n")
    
    # Show sample duplicates
    print("Sample duplicates:")
    for i, (forecast_date, horizon, count) in enumerate(duplicates[:5], 1):
        print(f"  {i}. {forecast_date} (horizon={horizon}): {count} copies")
    
    if len(duplicates) > 5:
        print(f"  ... and {len(duplicates) - 5} more")
    
    # Remove duplicates (keep earliest)
    total_removed = 0
    for forecast_date, horizon, count in duplicates:
        cursor = conn.execute("""
            DELETE FROM forecasts
            WHERE forecast_date = ? AND horizon = ?
            AND prediction_id NOT IN (
                SELECT prediction_id FROM forecasts
                WHERE forecast_date = ? AND horizon = ?
                ORDER BY timestamp ASC
                LIMIT 1
            )
        """, (forecast_date, horizon, forecast_date, horizon))
        
        removed = cursor.rowcount
        total_removed += removed
    
    conn.commit()
    conn.close()
    
    print(f"\n✅ Removed {total_removed} duplicate predictions")
    return total_removed


def show_stats():
    """Show database statistics."""
    conn = sqlite3.connect(DB_PATH)
    
    print("\n" + "=" * 80)
    print("DATABASE STATISTICS")
    print("=" * 80)
    
    # Total predictions
    cursor = conn.execute("SELECT COUNT(*) FROM forecasts")
    total = cursor.fetchone()[0]
    
    # With actuals
    cursor = conn.execute("""
        SELECT COUNT(*) FROM forecasts
        WHERE actual_vix_change IS NOT NULL
    """)
    with_actuals = cursor.fetchone()[0]
    
    # Date range
    cursor = conn.execute("""
        SELECT MIN(forecast_date) as earliest, MAX(forecast_date) as latest
        FROM forecasts
    """)
    earliest, latest = cursor.fetchone()
    
    # Remaining duplicates (should be 0 after cleanup)
    cursor = conn.execute("""
        SELECT COUNT(*) FROM (
            SELECT forecast_date, horizon, COUNT(*) as count
            FROM forecasts
            GROUP BY forecast_date, horizon
            HAVING count > 1
        )
    """)
    remaining_dupes = cursor.fetchone()[0]
    
    print(f"\nTotal predictions: {total}")
    print(f"  ✅ With actuals: {with_actuals}")
    print(f"  ⏳ Pending actuals: {total - with_actuals}")
    print(f"\nDate range: {earliest} to {latest}")
    print(f"Remaining duplicates: {remaining_dupes}")
    
    if remaining_dupes > 0:
        print(f"  ⚠️  WARNING: {remaining_dupes} duplicate pairs still exist!")
    else:
        print(f"  ✅ No duplicates")
    
    # Cohort breakdown
    print(f"\nPredictions by cohort:")
    df = pd.read_sql_query("""
        SELECT calendar_cohort, COUNT(*) as count
        FROM forecasts
        GROUP BY calendar_cohort
        ORDER BY count DESC
    """, conn)
    
    for _, row in df.iterrows():
        print(f"  {row['calendar_cohort']:25s}: {row['count']:4d}")
    
    conn.close()
    
    print("=" * 80 + "\n")


def main():
    """Run cleanup and show stats."""
    if not Path(DB_PATH).exists():
        print(f"❌ Database not found: {DB_PATH}")
        print(f"   Run batch forecasting first:")
        print(f"   python integrated_system_production.py --mode batch \\")
        print(f"     --start-date 2025-08-01 --end-date 2025-11-10")
        return
    
    # Remove duplicates
    removed = remove_duplicates()
    
    # Show final stats
    show_stats()
    
    if removed > 0:
        print("✅ Database cleaned successfully\n")


if __name__ == "__main__":
    main()


DUPLICATE CLEANUP
✅ No duplicates found

DATABASE STATISTICS

Total predictions: 75
  ✅ With actuals: 72
  ⏳ Pending actuals: 3

Date range: 2024-01-07T00:00:00 to 2025-11-10T00:00:00
Remaining duplicates: 0
  ✅ No duplicates

Predictions by cohort:
  mid_cycle                :   45
  monthly_opex_minus_1     :   11
  fomc_minus_3             :    7
  earnings_heavy           :    6
  monthly_opex_minus_5     :    6



In [65]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("data_cache/predictions.db")
df = pd.read_sql_query("""
    SELECT 
        forecast_date, 
        confidence_score, 
        current_vix, 
        quantile_coverage,
        created_at 
    FROM forecasts 
    ORDER BY forecast_date DESC
    LIMIT 50
""", conn)
print(df)
conn.close()

          forecast_date  confidence_score  current_vix  \
0   2025-11-05T00:00:00          0.939116    17.440001   
1   2025-11-04T00:00:00          0.938339    16.910000   
2   2025-11-03T00:00:00          0.943604    16.920000   
3   2025-11-02T00:00:00          0.849609    16.420000   
4   2025-11-01T00:00:00          0.849609    15.790000   
5   2025-10-29T00:00:00          0.849609    16.370001   
6   2025-10-28T00:00:00          0.849609    17.299999   
7   2025-10-27T00:00:00          0.801657    18.600000   
8   2025-10-26T00:00:00          0.801657    17.870001   
9   2025-10-25T00:00:00          0.798351    18.230000   
10  2025-10-22T00:00:00          0.471167    20.780001   
11  2025-10-21T00:00:00          0.471167    25.309999   
12  2025-10-20T00:00:00          0.471167    20.639999   
13  2025-10-19T00:00:00          0.754279    20.809999   
14  2025-10-18T00:00:00          0.751119    19.030001   
15  2025-10-15T00:00:00          0.754279    21.660000   
16  2025-10-14

In [64]:
# Quick database cleanup script
import sqlite3

db_path = "data_cache/predictions.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Delete predictions with missing actuals from your skeleton run
cursor.execute("""
    DELETE FROM forecasts 
    WHERE forecast_date >= '2024-01-01' 
    AND quantile_coverage IS NULL
""")

deleted = cursor.rowcount
conn.commit()
conn.close()

print(f"✅ Deleted {deleted} incomplete skeleton predictions")

✅ Deleted 0 incomplete skeleton predictions
